In [1]:

from __future__ import print_function
from __future__ import absolute_import
from __future__ import division
import csv
import pandas as pd
import os
import numpy as np
import augmentation as aug
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Lambda
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import Conv2D
from keras.layers import concatenate
from keras.layers import add
from keras.regularizers import l2
from keras.utils import get_file
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.applications.imagenet_utils import decode_predictions
from keras import backend as K

Using TensorFlow backend.
c:\users\xiang\.conda\envs\fianl_even\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\xiang\.conda\envs\fianl_even\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\xiang\.conda\envs\fianl_even\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\xiang\.conda\envs\fianl_even\lib

In [2]:
preprocess_input='preprocess_input'

__all__=['DualPathNetwork', 'DPN92', 'DPN98', 'DPN131', 'DPN107', 'preprocess_input', 'decode_predictions']

In [3]:
data_path= "C:/Users/xiang/Desktop/rsna-intracranial-hemorrhage-detection-128"
images_path=f'{data_path}/stage_2_train_images'
csv_path=f'{data_path}/stage_2_train.csv'
image_index=os.listdir(f'{data_path}/stage_2_train_images')
image_index=pd.DataFrame(image_index)
image_index=image_index[0].str.slice(stop=12).values

In [4]:
print(np.shape(image_index))

(755947,)


In [5]:
def shuffle_data(x):
    rand_indices=np.random.permutation(x.shape[0])
    train_indices=rand_indices[0:650000]
    vali_indices=rand_indices[650000:755940]
    return train_indices,vali_indices

In [6]:
gener=aug.DataGenerator(data_path,images_path=f'{data_path}/stage_2_train_images',csv_path=f'{data_path}/stage_2_train.csv',
                    num_thread=None,is_train=True)

tr_indices,vali_indices=shuffle_data(image_index)
print('train shape:',np.shape(tr_indices))
print('vali shape:',np.shape(vali_indices))

train shape: (650000,)
vali shape: (105940,)


In [7]:
def _initial_conv_block_inception(input, initial_conv_filters, weight_decay=5e-4):
    x = Conv2D(initial_conv_filters, (7, 7), padding='same', use_bias=False, kernel_initializer='he_normal',
               kernel_regularizer=l2(weight_decay), strides=(2, 2))(input)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    return x

In [8]:
def _bn_relu_conv_block(input, filters, kernel=(3, 3), stride=(1, 1), weight_decay=5e-4):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(filters, kernel, padding='same', use_bias=False, kernel_initializer='he_normal',
               kernel_regularizer=l2(weight_decay), strides=stride)(input)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    return x

In [9]:
def _grouped_convolution_block(input, grouped_channels, cardinality, strides, weight_decay=5e-4):
    init = input
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    group_list = []
    if cardinality == 1:
        # with cardinality 1, it is a standard convolution
        x = Conv2D(grouped_channels, (3, 3), padding='same', use_bias=False, strides=strides,
                   kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(init)
        x = BatchNormalization(axis=channel_axis)(x)
        x = Activation('relu')(x)
        return x
    for c in range(cardinality):
        x = Lambda(lambda z: z[:, :, :, c * grouped_channels:(c + 1) * grouped_channels]
                   if K.image_data_format() == 'channels_last' else
                   lambda z: z[:, c * grouped_channels:(c + 1) * grouped_channels, :, :])(input)

        x = Conv2D(grouped_channels, (3, 3), padding='same', use_bias=False, strides=strides,
                   kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(x)

        group_list.append(x)

    group_merge = concatenate(group_list, axis=channel_axis)
    group_merge = BatchNormalization(axis=channel_axis)(group_merge)
    group_merge = Activation('relu')(group_merge)
    return group_merge

In [10]:
def _dual_path_block(input, pointwise_filters_a, grouped_conv_filters_b, pointwise_filters_c,
                     filter_increment, cardinality, block_type='normal'):
    grouped_channels = int(grouped_conv_filters_b/ cardinality)
    init = concatenate(input, axis=-1) if isinstance(input, list) else input
    if block_type == 'projection':
        stride = (1, 1)
        projection = True
    elif block_type == 'downsample':
        stride = (2, 2)
        projection = True
    elif block_type == 'normal':
        stride = (1, 1)
        projection = False
    else:
        raise ValueError('`block_type` must be one of ["projection", "downsample", "normal"]. Given %s' % block_type)

    if projection:
        projection_path = _bn_relu_conv_block(init, filters=pointwise_filters_c + 2 * filter_increment,
                                              kernel=(1, 1), stride=stride)
        input_residual_path = Lambda(lambda z: z[:, :, :, :pointwise_filters_c]
                                     if K.image_data_format() == 'channels_last' else
                                     z[:, :pointwise_filters_c, :, :])(projection_path)
        input_dense_path = Lambda(lambda z: z[:, :, :, pointwise_filters_c:]
                                  if K.image_data_format() == 'channels_last' else
                                  z[:, pointwise_filters_c:, :, :])(projection_path)
    else:
        input_residual_path = input[0]
        input_dense_path = input[1]

    x = _bn_relu_conv_block(init, filters=pointwise_filters_a, kernel=(1, 1))
    x = _grouped_convolution_block(x, grouped_channels=grouped_channels, cardinality=cardinality, strides=stride)
    x = _bn_relu_conv_block(x, filters=pointwise_filters_c + filter_increment, kernel=(1, 1))

    output_residual_path = Lambda(lambda z: z[:, :, :, :pointwise_filters_c]
                                  if K.image_data_format() == 'channels_last' else
                                  z[:, :pointwise_filters_c, :, :])(x)
    output_dense_path = Lambda(lambda z: z[:, :, :, pointwise_filters_c:]
                               if K.image_data_format() == 'channels_last' else
                               z[:, pointwise_filters_c:, :, :])(x)

    residual_path = add([input_residual_path, output_residual_path])
    dense_path = concatenate([input_dense_path, output_dense_path], axis=-1)

    return [residual_path, dense_path]


In [11]:
def _create_dpn(nb_classes,img_input,include_top,initial_conv_filters,filter_increment,depth,cardinality=32,width=3,weight_decay=5e-4,pooling=None):
    N=list(depth)
    base_filters=256
    x=_initial_conv_block_inception(img_input,initial_conv_filters,weight_decay)

    filter_inc=filter_increment[0]
    filters=int(cardinality*width)

    for i in range(N[0]-1):
        x=_dual_path_block(x,pointwise_filters_a=filters,grouped_conv_filters_b=filters,pointwise_filters_c=base_filters,filter_increment=filter_inc,
                           cardinality=cardinality,block_type='projection')
    for i in range(N[0]-1):
        x = _dual_path_block(x, pointwise_filters_a=filters,
                             grouped_conv_filters_b=filters,
                             pointwise_filters_c=base_filters,
                             filter_increment=filter_inc,
                             cardinality=cardinality,
                             block_type='normal')
    for k in range(1, len(N)):
        print("BLOCK %d" % (k + 1))
        filter_inc = filter_increment[k]
        filters *= 2
        base_filters *= 2

        x = _dual_path_block(x, pointwise_filters_a=filters,
                             grouped_conv_filters_b=filters,
                             pointwise_filters_c=base_filters,
                             filter_increment=filter_inc,
                             cardinality=cardinality,
                             block_type='downsample')

        for i in range(N[k] - 1):
            x = _dual_path_block(x, pointwise_filters_a=filters,
                                 grouped_conv_filters_b=filters,
                                 pointwise_filters_c=base_filters,
                                 filter_increment=filter_inc,
                                 cardinality=cardinality,
                                 block_type='normal')

    x = concatenate(x, axis=-1)
    if include_top:
        avg = GlobalAveragePooling2D()(x)
        max = GlobalMaxPooling2D()(x)
        x = add([avg,max])
        x = Lambda(lambda z: 0.5 * z)(x)
        x = Dense(nb_classes, use_bias=False,kernel_regularizer=l2(weight_decay),
                  kernel_initializer='he_normal', activation='sigmoid')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
            x= Dense(units=nb_classes,
                 activation='sigmoid',
                 kernel_initializer='he_normal',
                 kernel_regularizer=l2(weight_decay))(x)

        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)
        elif pooling == 'max-avg':
            a = GlobalMaxPooling2D()(x)
            b = GlobalAveragePooling2D()(x)
            x = add([a, b])
            x = Lambda(lambda z: 0.5 * z)(x)

    return x

In [12]:
def DualPathNetwork(input_shape=None,initial_conv_filters=64,depth=[3, 4, 20, 3],filter_increment=[16, 32, 24, 128],
                    cardinality=32,width=3,weight_decay=0,include_top=True,weights=None,input_tensor=None,pooling=None,classes=6):
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')
    assert len(depth)==len(filter_increment)
    input_shape=_obtain_input_shape(input_shape,default_size=224,min_size=28,data_format=K.image_data_format(),require_flatten=include_top)

    if input_tensor is None:
        img_input=Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input=Input(tensor=input_tensor,shape=input_shape)
        else:
            img_input=input_tensor
    x = _create_dpn(classes, img_input, include_top, initial_conv_filters,
                    filter_increment, depth, cardinality, width, weight_decay, pooling)
    if input_tensor is not None:
        inputs=get_source_inputs(input_tensor)
    else:
        inputs=img_input

    model=Model(inputs,x,name='DPN')

    return model

In [13]:
def DPN92(input_shape=None,
          include_top=True,
          weights=None,
          input_tensor=None,
          pooling=None,
          classes=6):
    return DualPathNetwork(input_shape, include_top=include_top, weights=weights, input_tensor=input_tensor,
                           pooling=pooling, classes=classes)


def DPN98(input_shape=None,
          include_top=True,
          weights=None,
          input_tensor=None,
          pooling=None,
          classes=6):
    return DualPathNetwork(input_shape, initial_conv_filters=96, depth=[3, 6, 20, 3], filter_increment=[16, 32, 32, 128],
                           cardinality=40, width=4, include_top=include_top, weights=weights, input_tensor=input_tensor,
                           pooling=pooling, classes=classes)


def DPN131(input_shape=None,
           include_top=True,
           weights=None,
           input_tensor=None,
           pooling=None,
           classes=6):
    return DualPathNetwork(input_shape, initial_conv_filters=128, depth=[4, 8, 28, 3], filter_increment=[16, 32, 32, 128],
                           cardinality=40, width=4, include_top=include_top, weights=weights, input_tensor=input_tensor,
                           pooling=pooling, classes=classes)


def DPN107(input_shape=None,
           include_top=True,
           weights=None,
           input_tensor=None,
           pooling=None,
           classes=6):
    return DualPathNetwork(input_shape, initial_conv_filters=128, depth=[4, 8, 20, 3], filter_increment=[20, 64, 64, 128],
                           cardinality=50, width=4, include_top=include_top, weights=weights, input_tensor=input_tensor,
                           pooling=pooling, classes=classes)


In [14]:
image_size=128
iterations=10000
img_rows, img_cols = 128,128
img_channels = 1
num_classes= 6
batch_size=25        
epochs=20
weight_decay= 1e-4

In [15]:

model =DPN92((img_rows,img_cols,1),include_top=False,classes=6,pooling='avg')
model.summary()

BLOCK 2
BLOCK 3
BLOCK 4
Model: "DPN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   3136        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 64)   0           batch_normalization_1[0][0]      
________________________________________________________________________

In [16]:
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
# from keras.preprocessing.image import ImageDataGenerator
# img_input = Input(shape=(img_rows, img_cols, img_channels))
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['binary_accuracy'])
# reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,patience=10, min_lr=0.001)
checkpointer=ModelCheckpoint(filepath=os.path.join(data_path,'chack_point_weight.hdf5'),save_best_only=True,save_weights_only=True)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
model.load_weights(os.path.join(data_path,'my_model_weights.h5'))

In [18]:
history=model.fit_generator(generator=gener.get_X_batch(image_index[tr_indices],batch_size,image_size,True),steps_per_epoch=iterations,epochs=epochs,callbacks=None,
                            validation_data=gener.get_X_batch(image_index[vali_indices],20,image_size,False),validation_steps=len(vali_indices)//20,class_weight={0:2.,1:1.,2:1.,3:1.,4:1.,5:1.})
print(history)

Epoch 1/20
10000/10000 [==============================] - 10533s 1s/step - loss: 0.5861 - binary_accuracy: 0.8841 - val_loss: 0.2654 - val_binary_accuracy: 0.8875
Epoch 2/20
10000/10000 [==============================] - 10436s 1s/step - loss: 0.5853 - binary_accuracy: 0.8843 - val_loss: 0.3838 - val_binary_accuracy: 0.8629
Epoch 3/20
10000/10000 [==============================] - 10431s 1s/step - loss: 0.5850 - binary_accuracy: 0.8844 - val_loss: 0.1625 - val_binary_accuracy: 0.8870
Epoch 4/20
10000/10000 [==============================] - 10425s 1s/step - loss: 0.5857 - binary_accuracy: 0.8842 - val_loss: 0.2382 - val_binary_accuracy: 0.8703
Epoch 5/20
10000/10000 [==============================] - 10415s 1s/step - loss: 0.5827 - binary_accuracy: 0.8851 - val_loss: 0.2923 - val_binary_accuracy: 0.8853
Epoch 6/20
10000/10000 [==============================] - 10408s 1s/step - loss: 0.5830 - binary_accuracy: 0.8848 - val_loss: 0.2387 - val_binary_accuracy: 0.8701
Epoch 7/20
10000/10000

In [19]:

test_image_path=f'{data_path}/stage_2_test_images'
test_image_index=os.listdir(f'{data_path}/stage_2_test_images')
test_image_index=pd.DataFrame(test_image_index)
test_image_index=test_image_index[0].str.slice(stop=12).values




In [20]:
from concurrent.futures import ThreadPoolExecutor as threadPool
from PIL import Image
def load_image(temp_path):
        im = np.array(Image.open(test_image_path+'/'+temp_path+'.png'))
#         print(im)
        return im
        
def get_img(img_paths, img_size):
        p = threadPool()
        X = np.zeros((len(img_paths), img_size, img_size), dtype=np.uint8)
        i = 0
        for future in p.map(load_image, img_paths):
            img = np.resize(future, (img_size, img_size))
#             print(img)
            X[i, :, :] = img
            
            i += 1
        p.shutdown(wait=False)
        return X

In [21]:
def get_test_batch(X_path,batch_size,img_size):
        try:
            if len(X_path) %batch_size != 0:
                raise Exception("batchSize not match the size of data!")
        except Exception as err:
            print(err)
        else:
            while 1:
                for i in range(0, len(X_path), batch_size):
                    X =(get_img(X_path[i:i + batch_size], img_size))
                    yield np.resize(X/255,(len(X),img_size,img_size,1)).astype('float16')

In [22]:
test_gener=get_test_batch(test_image_index,16,128)

In [23]:
submission_result=model.predict_generator(generator=test_gener,steps=len(test_image_index)//16)
print(submission_result)

[[2.05431521e-01 9.48023796e-03 9.07397270e-03 1.95831060e-04
  3.24777067e-02 1.41400844e-01]
 [1.64663792e-03 6.85453415e-07 1.40070915e-05 8.64267349e-07
  7.53790140e-04 3.82810831e-04]
 [5.88020682e-02 1.93083286e-03 1.04486942e-02 2.00161338e-03
  4.49567139e-02 1.39557123e-02]
 ...
 [2.68132687e-02 2.02324986e-03 4.37593460e-03 9.97781754e-05
  9.87571478e-03 7.34478235e-03]
 [8.90582800e-03 1.29044056e-05 7.40289688e-05 3.57627869e-07
  1.02296472e-03 3.69477272e-03]
 [3.92246485e-01 5.73506057e-02 6.20033741e-02 2.65708566e-03
  1.43832713e-01 2.57959127e-01]]


In [24]:
def evaluation(prob):        
    return max(min(prob,1-pow(10,-15)),pow(10,-15))

In [25]:
for i in range(len(submission_result)):
    for j in range(len(submission_result[i])):
        submission_result[i,j]=evaluation(submission_result[i,j])

In [26]:
def read_csv(filename):
            df = pd.read_csv(filename)
            df["ImageID"] = df["ID"].str.slice(stop=12)
            df["Diagnosis"] = df["ID"].str.slice(start=13)
            df = df.loc[:, ["Label", "Diagnosis", "ImageID"]]
            df = df.set_index(['ImageID', 'Diagnosis']).unstack(level=-1)
            return df

In [27]:
submission_csv=read_csv( 'C:/Users/xiang/Desktop/rsna-intracranial-hemorrhage-detection-128/stage_2_sample_submission.csv')
submission_csv.head()

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
ImageID                                                                      
ID_000000e27   0.5      0.5              0.5              0.5          0.5   
ID_000009146   0.5      0.5              0.5              0.5          0.5   
ID_00007b8cb   0.5      0.5              0.5              0.5          0.5   
ID_000134952   0.5      0.5              0.5              0.5          0.5   
ID_000176f2a   0.5      0.5              0.5              0.5          0.5   

                       
Diagnosis    subdural  
ImageID                
ID_000000e27      0.5  
ID_000009146      0.5  
ID_00007b8cb      0.5  
ID_000134952      0.5  
ID_000176f2a      0.5

In [28]:
result_dict={}
for i in range(len(test_image_index)):
    result_dict[test_image_index[i]]=submission_result[i]

In [29]:
for i in submission_csv.index:
    submission_csv.loc[i]=result_dict[i]

In [30]:
submission_csv = submission_csv.stack().reset_index()
submission_csv.insert(loc=0, column='ID', value=submission_csv['ImageID'].astype(str) + "_" + submission_csv['Diagnosis'])
submission_csv= submission_csv.drop(["ImageID", "Diagnosis"], axis=1)
submission_csv.to_csv('stage_2_sample_DPN3.csv', index=False)
submission_csv.head()

,ID,Label
0,ID_000000e27_any,0.205432
1,ID_000000e27_epidural,0.009480
2,ID_000000e27_intraparenchymal,0.009074
3,ID_000000e27_intraventricular,0.000196
4,ID_000000e27_subarachnoid,0.032478


In [31]:
model.save_weights(os.path.join(data_path,'my_model_weights.h5'))
model.save(os.path.join(data_path,'my_model.h5'))